In [1]:
import git
import os
repo_url = 'https://github.com/sahajsoft/Pravarthan'
# clone the repo if it doesn't exist
expanded_path = os.path.expanduser('~/Pravarthan')

if not os.path.exists(expanded_path):
    repo = git.Repo.clone_from(repo_url, expanded_path)
else:
    repo = git.Repo(expanded_path)    


In [2]:
from llama_index.core import VectorStoreIndex, Document
from llama_index.core.node_parser import SentenceSplitter
import chardet
import os
from pygments.lexers import guess_lexer_for_filename
from pygments.util import ClassNotFound
import sys
sys.path.append("..")

from libraries.utils.io import scan_dir

# Recursively scan the directory and return
# a dictionary of valid script files along with
# their programming language
language_map = scan_dir(expanded_path, verbose=True)

Progress: 1032/1032 | Scanning: /Users/joey/Pravarthan/base/templates/base/contract_list.html                                        
The following number of files with the specified file extensions were ignored: 
.xlsx: 8
: 9
.baseimage: 1
.png: 27
.py: 51
.gif: 51
.woff: 9
.eot: 6
.woff2: 5
.ttf: 22
.svg: 26
.conf: 4
.swp: 1
.py~HEAD: 1
.2020-01-01: 1
.ico: 1
.csv: 3


In [3]:
from libraries.utils.helpers import get_formatted_dict_key_counts

print("The following languages were detected with their respective file counts")
print(get_formatted_dict_key_counts(language_map))

The following languages were detected with their respective file counts

Bash: 11
Python: 494
Text only: 17
YAML: 9
Docker: 1
Markdown: 2
INI: 2
CSS: 38
SCSS: 4
LessCss: 4
JavaScript: 78
HTML: 131
Systemd: 3
JSON: 10
Transact-SQL: 3


In [4]:
from libraries.llama_helpers.code_splitter import apply_code_splitters, get_language_variable_code_splitters

code_splitters = get_language_variable_code_splitters(language_map.keys())

nodes = apply_code_splitters(code_splitters, language_map)
print(len(nodes))

Could not get parser for language text only. Check https://github.com/grantjenks/py-tree-sitter-languages#license for a list of valid languages.
Could not get parser for language docker. Check https://github.com/grantjenks/py-tree-sitter-languages#license for a list of valid languages.
Could not get parser for language ini. Check https://github.com/grantjenks/py-tree-sitter-languages#license for a list of valid languages.
Could not get parser for language scss. Check https://github.com/grantjenks/py-tree-sitter-languages#license for a list of valid languages.
Could not get parser for language lesscss. Check https://github.com/grantjenks/py-tree-sitter-languages#license for a list of valid languages.
Could not get parser for language systemd. Check https://github.com/grantjenks/py-tree-sitter-languages#license for a list of valid languages.
Could not get parser for language transact-sql. Check https://github.com/grantjenks/py-tree-sitter-languages#license for a list of valid languages.


/Users/joey/Library/Caches/pypoetry/virtualenvs/coding-assistant-JkLEjshP-py3.12/lib/python3.12/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


Getting nodes for HTML: 131 files
Getting nodes for JSON: 10 files
9718


In [5]:
# Check nodes
print("========== Printing node 0\n")
print(nodes[0])

print("\n\n========== Printing node 1500\n")
print(nodes[1500])

========== Printing node 0

dir_names=""
for i in $(ls -d */);
    do
        if [ -f "$i/freeze.txt" ]
        then
            if [ `cat "$i/freeze.txt" | grep "^migrated$"` ]
            then
                dir_names="$dir_names${i%?} "
            fi
        fi
    done;
echo $dir_names


========== Printing node 1500

class ProjectsStaffing(ListModelMixin,
                       generics.GenericAPIView):
    serializer_class = ProjectStaffingSerializer

    def get_queryset(self):
        return ProjectStaffing.objects.filter(id=self.kwargs['pk'])

    def get(self, request, *args, **kwargs):
        return self.list(request, *args, **kwargs)


class ProjectDetails(RetrieveUpdateAPIView):
    queryset = Project.objects.all()
    serializer_class = project_serializer.ProjectSerializer
    permission_classes = [ResourceOwnerAccessPermission]


class ReturnAllCountryOffice(GenericViewSet, ListModelMixin):
    queryset = CountryOffice.objects.all().order_by('id')
    serializer_class

In [ ]:
import ollama
import weaviate
import weaviate.classes.config as wc
import weaviate.classes as wvc
import sys
from libraries.embeddings import gen_embeddings
from libraries.weaviate_client import create_or_get_collection, apply_embeddings, local_client

embedding_model = "nomic-embed-text"

client = local_client()
collection = create_or_get_collection("test")

embeddings = apply_embeddings(gen_embeddings, embedding_model, nodes)



Loading... 36.20086437538588%%%%

In [ ]:
print(client.is_ready())
print(client.is_connected())

res = collection.data.insert_many(embeddings)


In [ ]:
retrieved_docs = collection.query.near_vector(
    near_vector=gen_embeddings('''How can I get all active members''', embedding_model),
    limit=15,
    return_metadata=wvc.query.MetadataQuery(certainty=True)
)

retrieved_messages = [o.properties['message'] for o in retrieved_docs.objects]

print(retrieved_messages[1])